## Overview

This notebook has 4 major sections. If you want to doive into a partiuclar section - plecase use the below links -
1. [Data Cleaning](#1.-Data-Cleaning)
3. [Data Analysis](#2.-Data-Analysis)
4. [Defining bot clicks](#3.-Defining-bot-clicks)
5. [Building ML model](#4.-Building-ML-model)

### Import packages

In [ ]:
import pandas as pd
import urllib.parse
import json
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import duckdb
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    precision_recall_curve,
    roc_curve,
    auc
)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# 1. Data Cleaning

## Clean url column

In [ ]:
raw_data = pd.read_csv('bot-hunter-dataset.tsv', sep='\t', header=None, names=['date_time', 'server_location', 'browser', 'device', 'url', 'empty_column'])
raw_data.drop('empty_column', axis=1, inplace=True)
raw_data['json_url']=raw_data.apply(lambda x: urllib.parse.parse_qs(urllib.parse.urlparse(x['url']).query), axis=1)

In [ ]:
raw_data.head()

## No Na's in raw data

In [ ]:
raw_data.isna().sum()

### These are all the possible attributes in url- we will define them below in separate columns

In [ ]:
def get_unique_json_keys(df, col):
    all_keys = Counter()
    for val in df[col]:
        all_keys.update(val.keys())
    return all_keys

get_unique_json_keys(raw_data, 'json_url')

In [ ]:
# source of param names - https://duckduckgo.com/duckduckgo-help-pages/settings/params
def clean_url(url: str) -> dict:
    parsed = urllib.parse.urlparse(url)
    params = urllib.parse.parse_qs(parsed.query)
    params = {k: v[0] for k, v in params.items()}

    mapped = {
        "n": params.get("n"),
        "clicked_ad": params.get("d"),
        "slot_id": params.get("sld"),
        "source_type": params.get("st"),
        "network_type": params.get("nt"),
        "r": params.get("r"),
        "ad_exchange": params.get("adx"),
        "ad_exchange_name": params.get("adx_name"),
        "time_to_click": params.get("ttc"),
        "search_term": params.get("q"),
        "country": params.get("ct"),
        "region": params.get("kl"),
        "safe_search": params.get("kp"),
        "f": params.get("f"),
        "bkl": params.get("bkl"),
        "lsexp1": params.get("lsexp1"),
        "om": params.get("om"),
        "atb": params.get("atb"),
        "atbva": params.get("atbva"),
        "va": params.get("va"),
        "atbexp": params.get("atbexp"),
}
    return mapped

raw_data['clean_json_url']=raw_data.apply(lambda x: clean_url(x['url']), axis=1)

In [ ]:
data = pd.concat([raw_data[['date_time', 'server_location', 'browser', 'device']], pd.json_normalize(raw_data['clean_json_url'])],axis = 1) 

In [ ]:
data.drop_duplicates(inplace=True)

## Clean data

In [ ]:
data.head()

# 2. Data Analysis

## EDA
1. There are several parameters in the URL such as r, f, bkl, lspexp1, om, atb, atbva, va, and atbexp—for which I couldn’t find clear definitions. Given the time constraints, I will not explore these further as part of this project.
2. All data is from mobile device
3. Interesting wt-wt is the most common region which stands for "No region"
4. Most traffic is from android device
5. Only 1 % traffic has safe search ON
6. All data is for a single day: 2019-12-02

In [ ]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['time_to_click'] = data['time_to_click'].astype('int')

In [ ]:
print(f'The min time period {data.date_time.min()} and max is {data.date_time.max()}')

In [ ]:
data.country.value_counts(normalize=True)

In [ ]:
data.isna().sum()

In [ ]:
data.clicked_ad.value_counts()

In [ ]:
def plot_value_counts(df):
    """
    Plots value counts for each column in the DataFrame.
    """
    for col in df.columns:
        if col not in ['date_time', 'clicked_ad', 'time_to_click', 'search_term', 'country']:
            plt.figure(figsize=(8, 4))
            df[col].value_counts(dropna=False).plot(kind='bar')
            plt.title(f"Value Counts for '{col}'")
            plt.xlabel("Value")
            plt.ylabel("Count")
            plt.tight_layout()
            plt.show()

plot_value_counts(data)

In [ ]:
data.safe_search.value_counts(normalize=True)

## Region vs Time to click histogram

In [ ]:
plt.figure(figsize=(15, 4))
plt.title('Region vs Time to click (Only regions with atleast 100 clicks)')
plt.xticks(rotation=90)
sns.boxplot(x=data[data.region.isin(data['region'].value_counts()[(data['region'].value_counts()>100)].index)].region, 
            y=data[data.region.isin(data['region'].value_counts()[(data['region'].value_counts()>100)].index)].time_to_click)

In [ ]:
data["time_to_click"].quantile([i/10 for i in range(0,11)])

## Time of hour vs time to click

In [ ]:
fig, axs = plt.subplots(nrows=8, ncols=3, sharex=True, sharey = True, figsize=(18,10))
axs = axs.flatten()

for i in range(24):
    low, high = 0, np.percentile(data[data.date_time.dt.hour == i]["time_to_click"], 98)
    mean_val = data[data.date_time.dt.hour == i]["time_to_click"].clip(low,high).mean()
    std_val = data[data.date_time.dt.hour == i]["time_to_click"].clip(low,high).std()
    count_val = data[data.date_time.dt.hour == i]["time_to_click"].clip(low,high).shape[0]
    percentiles = np.percentile(data[data.date_time.dt.hour == i]["time_to_click"].clip(low,high), [1, 90, 99])
    sns.kdeplot(data[data.date_time.dt.hour == i]['time_to_click'].clip(low,high), ax=axs[i])
    axs[i].set_title(f'Time to click kde plot for Hour {i}')
    axs[i].set_xlim(0, high)
    
    axs[i].text(
    0.95, 0.85,
    f"μ={mean_val:.0f}\nσ={std_val:.0f}\ncount={count_val:.0f}",
    transform=axs[i].transAxes,
    ha="right", va="top",
    fontsize=10, color="black",
    bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.2")
    )

    for p, val in zip([1, 90, 99], percentiles):
        axs[i].axvline(val, color="red", linestyle="--", alpha=0.6)
        axs[i].text(val, plt.ylim()[1]*0.9, f"{p}%", rotation=90, va='top', ha='center', fontsize=8, color='red')

#plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.1, hspace=0.35)
plt.tight_layout()
plt.subplots_adjust(wspace=0.1)

In [ ]:
plt.figure(figsize=(10,4))
plt.ylabel('Number of clicks')
plt.xlabel('Hour of day')
plt.xticks(range(24))
plt.title("Number of clicks per hour")
plt.plot(data[['date_time']].groupby(data.date_time.dt.hour).count(), marker = 'o')

## Click heatmap

In [ ]:
plt.figure(figsize=(6,10))
data['date_time_floor'] = data['date_time'].dt.floor('min')
heatmap_data = data[data.search_term.isin(data.search_term.value_counts()[:30].index.to_list())].groupby(['search_term','date_time_floor']).size().unstack(fill_value=0)
heatmap_data = heatmap_data.loc[heatmap_data.sum(axis=1).sort_values(ascending=False).index]

sns.heatmap(heatmap_data, cmap='mako', cbar_kws={'label': 'Clicks'})
plt.xticks([])
plt.title(f"Click Intensity Heatmap by Minute for top 30 search terms")

# 3. Defining bot clicks

## Identify bot clicks
1. More than 5 clicks for same search terms and clicked ad with max difference between each click being less than 300 seconds
2. Find clicks that are ultra low time to click (Assume less than 1.5s)

In [ ]:
fast_time_to_click = duckdb.query(
    """
    select clicked_ad, search_term, date_time, time_to_click
    from data
    where time_to_click < 1500
    """
).to_df()

fast_time_to_click['label'] = 1

In [ ]:
bursts = duckdb.query(
    """
    WITH ordered AS (
          SELECT
            search_term,
            clicked_ad,
            date_time,
            time_to_click,
            ROW_NUMBER() OVER (PARTITION BY search_term, clicked_ad ORDER BY date_time) AS rn,
            LAG(date_time) OVER (PARTITION BY search_term, clicked_ad ORDER BY date_time) AS time_lag
          FROM data
),
tagged AS (
          SELECT
            search_term,
            clicked_ad,
            date_time,
            time_lag,
            time_to_click,
            rn,
            date_diff('second', time_lag, date_time) as time_diff,
            SUM(CASE
              WHEN time_lag IS NULL OR date_diff('second', time_lag, date_time) <= 300
                THEN 1 ELSE 0
            END) OVER (PARTITION BY search_term, clicked_ad ORDER BY date_time) AS click_count
          FROM ordered
),
grouped AS (
          SELECT
            tagged.*,
            rn - click_count AS grp_id
          FROM tagged
),
bursts AS (
    SELECT
            search_term,
            clicked_ad,
            grp_id,
            count(*) as streak
          FROM grouped
          WHERE time_diff <= 300
          GROUP BY search_term,
            clicked_ad,
            grp_id
            HAVING count(*) >= 5
)

SELECT g.search_term, g.clicked_ad, g.date_time, g.time_to_click
FROM grouped g
JOIN bursts b
  ON g.search_term = b.search_term
  AND g.clicked_ad = b.clicked_ad
  AND g.grp_id = b.grp_id
ORDER BY g.search_term, g.clicked_ad, g.date_time;
    """
).to_df()

bursts['label'] = 1

# 4. Building ML model

## Create training data 

In [ ]:
cols = ['date_time', 'search_term', 'clicked_ad', 'server_location', 'browser', 'device', 'slot_id', 'country', 'safe_search', 'time_to_click']
df = data[cols]
df.dropna(inplace=True)

label = pd.concat([bursts, fast_time_to_click]).reset_index(drop=True).drop_duplicates()

df = df.merge(label, on=['clicked_ad', 'search_term', 'date_time', 'time_to_click'], how='left').fillna(0)

## this feature can cause leakage as label is based off that and we would not have time to click information at the time our model runs to evaluate bots
df.drop('time_to_click', axis = 1, inplace = True)

In [ ]:
df.shape

## Feature engineering

In [ ]:
## create features from search term
df['term_length'] = df['search_term'].str.len()
df['num_words'] = df['search_term'].str.split().str.len()
df['has_numbers'] = df['search_term'].str.contains(r'\d').astype(int)
df['term_freq'] = df.groupby('search_term')['search_term'].transform('count')

## create features from date time
df['hour_of_day'] = df['date_time'].dt.hour

## type of features
text_features = ['search_term', 'clicked_ad']
cat_features = ['server_location', 'browser', 'device', 'country', 'safe_search', 'hour_of_day', 'slot_id']
num_features = ['term_length', 'num_words', 'has_numbers', 'term_freq']

## Train test split

In [ ]:
X = df[text_features + cat_features + num_features]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

## Encoding features

In [ ]:
## Text: TF-IDF
text_transformer = ColumnTransformer(
    transformers=[
        ('search_tfidf', TfidfVectorizer(max_features=500, ngram_range=(1,2)), 'search_term'),
        ('clicked_ad_tfidf', TfidfVectorizer(max_features=500, ngram_range=(1,2)), 'clicked_ad')
    ],
    remainder='drop'
)

## Categorical: One-hot encoding
cat_transformer = OneHotEncoder(handle_unknown='ignore')

## Numeric: Use directly (scaling optional)
num_transformer = StandardScaler()

# -------------------------------------------------
# Combine them in a ColumnTransformer
# -------------------------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_features),
        ('cat', cat_transformer, cat_features),
        ('num', num_transformer, num_features)
    ],
    remainder='drop'   # any extra columns are ignored
)

## Training Model

In [ ]:
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'  # helpful for imbalanced bot vs human
    ))
])

# -------------------------------------------------
# Train model
# -------------------------------------------------
rf_model.fit(X_train, y_train)

## Evaluate performance

In [ ]:
y_pred = rf_model.predict(X_test)
y_scores = rf_model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
plt.figure(figsize=(6, 4))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve (Random Forest)')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC Curve (Random Forest)')
plt.grid(True)
plt.show()

In [ ]:
target_precision = 0.9
idx = np.argmin(np.abs(precision - target_precision))
best_threshold = thresholds[idx]
print(f"Best threshold for precision ≈ {target_precision} : {round(best_threshold, 4)}")
print("Precision:", precision[idx])
print("Recall:", recall[idx])